In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

sns.set_theme()

from lab2 import Helper


from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

from sklearn.metrics import accuracy_score

In [2]:
df1 = pd.read_csv('Data/df1.csv')
df2 = pd.read_csv('Data/df2.csv')

display(df1.head(3), df2.head(3))

,id,age,cholesterol,gluc,smoke,alco,active,cardio,gender_2,bmi-feature_obese (class I),bmi-feature_obese (class II),bmi-feature_obese (class III),bmi-feature_overweight,bp-feature_healthy,bp-feature_hypertension crises,bp-feature_stage 1 hypertension,bp-feature_stage 2 hypertension
0,0,18393,1,1,0,0,1,0,1,0,0,0,0,0,0,1,0
1,1,20228,3,1,0,0,1,1,0,1,0,0,0,0,0,0,1
2,2,18857,3,1,0,0,0,1,0,0,0,0,0,0,0,1,0


,id,age,bmi,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,gender_2
0,0,18393,21,110,80,1,1,0,0,1,0,1
1,1,20228,34,140,90,3,1,0,0,1,1,0
2,2,18857,23,130,70,3,1,0,0,0,1,0


In [3]:
# Drop id as its not useful
df1 = df1.drop('id', axis=1)
df2 = df2.drop('id', axis=1)

### 2.4 - Välja modell

Chosen models:
* Logistic regression
* decision tree
* random forest

##### train|validation|test split

In [4]:
df1_x, df1_y = df1.drop('cardio', axis=1), df1['cardio']
df2_x, df2_y = df2.drop('cardio', axis=1), df2['cardio']

display(
    df1_x.head(1),
    df2_x.head(1)
)

,age,cholesterol,gluc,smoke,alco,active,gender_2,bmi-feature_obese (class I),bmi-feature_obese (class II),bmi-feature_obese (class III),bmi-feature_overweight,bp-feature_healthy,bp-feature_hypertension crises,bp-feature_stage 1 hypertension,bp-feature_stage 2 hypertension
0,18393,1,1,0,0,1,1,0,0,0,0,0,0,1,0


,age,bmi,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,gender_2
0,18393,21,110,80,1,1,0,0,1,1


In [5]:
df1_x_train, df1_x_val, df1_x_test, df1_y_train, df1_y_val, df1_y_test = Helper.train_val_test_split(df1_x, df1_y, split_size=0.2, rand_state=42)
df2_x_train, df2_x_val, df2_x_test, df2_y_train, df2_y_val, df2_y_test = Helper.train_val_test_split(df2_x, df2_y, split_size=0.2, rand_state=42)

display(
    (df1_x_train.shape, df1_y_train.shape, df1_x_val.shape, df1_y_val.shape, df1_x_test.shape, df1_y_test.shape),
    (df2_x_train.shape, df2_y_train.shape, df2_x_val.shape, df2_y_val.shape, df2_x_test.shape, df2_y_test.shape),
)

((40918, 15), (40918,), (13640, 15), (13640,), (13640, 15), (13640,))

((40918, 10), (40918,), (13640, 10), (13640,), (13640, 10), (13640,))

In [6]:
df1_x_train, df1_x_val = Helper.scaler('minmax', df1_x_train, df1_x_val)
df1_x_train.mean(), df1_x_train.std(), df1_x_val.mean(), df1_x_val.std()

(0.24469073517266024,
 0.4111901139656925,
 0.24653791612942894,
 0.4122246262036306)

## Grid Search parameters
- Scoring: 'Accuracy'
    - Reason: Given that where using classification, and not regression based models, accuracy seems like the only appropriate metric, given it directly tells us how good the model's doing at actually classifying things correctly.
- cv: 5
    - Reason: After searching a bit online, it seems as though 5 is a generally good number of folds for avoiding things like overfitting.

### LogisticRegression
- Chosen parameters
    - penalty: 'elasticnet'
        - Reason: We'd like to use both L1 and L2 regularization in the grid search
    
    - solver: saga
        - Reason: The only solver that supports elasticnet
    
    - max_iter: 100 (Default)
        - Reason: Appropriate number of interations for convergence.

- Hyperparameters
    - l1_ratio: [0.0, 0.1, ... 1.0]
        - Reason: Uses both solvers individually as well as in ratios with a good increment resolution of 0.1.
    
### DecisionTreeClassifier
- Chosen parameters
    - max_depth: None (Default)
        - Reason:

    - min_samples_split: 2 (Default)
        - Reason:

    - min_samples_leaf: 1 (Default)
        - Reason:

    - min_weight_fraction_leaf: 0 (Default)
        - Reason:

    - max_leaf_nodes: None (Default)
        - Reason:

    - min_impurity_decrease: 0 (Default)
        - Reason:

    - class_weight: None (Default)
        - Reason:

    - ccp_alpha: 0 (Default)
        - Reason:

- Hyperparameters
    - criterion ['gini', 'entropy', 'log_loss']
    - splitter ['best', 'random']
    - max_features: [None, 'sqrt', 'log2']


### RandomForestClassifier
- Chosen parameters
    - max_depth: None (Default)
        - Reason:

    - min_samples_split: 2 (Default)
        - Reason:

    - min_samples_leaf: 1 (Default)
        - Reason:
    
    - min_weight_fraction_leaf: 0 (Default)
        - Reason:

    - max_leaf_nodes: None (Default)
        - Reason:
    
    - min_impurity_decrease: 0 (Default)
        - Reason:

    - bootstrap: True (Default)
        - Reason:
        
    - oob_score: False (Default)
        - Reason:

    - class_weight: None (Default)
        - Reason:
    
    - ccp_alpha: None (Default)
        - Reason:

- Hyperparameters
    - n_estimators: [10, 20, ... 100] 
        - Reason:
    
    - criterion ['gini', 'entropy', 'log_loss']
    - max_features: [None, 'sqrt', 'log2']
    
    - max_samples: [None, 0.1, 0.2, ... 1.0] # would this actually work?
        - Reason:

In [30]:
test = [float(x)/10 for x in range(1, 10)]
test

[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [15]:
# All the data needed for chosing the right model and doing hyperparameter tuning via GridSearchCV

datasets = {
    'df1': {
        'x_train': df1_x_train,
        'x_val': df1_x_val,
        'x_test': df1_x_test,
        'y_train': df1_y_train,
        'y_val': df1_y_val,
        'y_test': df1_y_test
    },
    'df2': {
        'x_train': df2_x_train,
        'x_val': df2_x_val,
        'x_test': df2_x_test,
        'y_train': df2_y_train,
        'y_val': df2_y_val,
        'y_val': df2_y_test
     }
}


model_data = {
    'LogisticRegression': {
        'model': LogisticRegression(),
        'search space': {
            # Chosen parameters (defaults excluded)
            'penalty': ['elasticnet'],
            'solver': ['saga'],
            
            # Hyperparamaters
            'l1_ratio' : [round(x * 0.1, 1) for x in range(11)]
        }
    },
    
    'DecisionTreeClassifier': {
        'model': DecisionTreeClassifier(),
        'search space': {
            # Chosen parameters (defaults excluded)
            
            # Hyperparamaters
            'criterion': ['gini', 'entropy', 'log_loss'],
            'splitter': ['best', 'random'],
            'max_features' : [None, 'sqrt', 'log2'],
        }
    },
    
    'RandomForestClassifier': {
        'model': RandomForestClassifier(),
        'search space': {
            # Chosen parameters
            
            # Hyperparamaters
            'n_estimators': [x for x in range(10, 110, 10)],
            
            'criterion': ['gini', 'entropy', 'log_loss'],
            'max_features' : [None, 'sqrt', 'log2']
        }
    },
}

In [9]:
model_metrics = pd.DataFrame(columns=['Dataset', 'Scaling', 'Model', 'Hyper params', 'Accuracy'])
model_metrics

,Dataset,Scaling,Model,Hyper params,Accuracy


In [14]:
import warnings
warnings.filterwarnings('ignore')

model_metrics = pd.DataFrame(columns=['Dataset', 'Scaling', 'Model', 'Hyper params', 'Accuracy'])
model_metrics

for dataset_name, dataset in datasets.items():
    
    # Loop through and use the minmax and standard scaler
    for scaler_name in ['minmax', 'standard']:
        
        # Scale the data
        scaled_x_train, scaled_x_val = Helper.scaler(scaler_name, dataset['x_train'], dataset['x_val'])

        # loop through all models. data = dict with model object and parameter info
        for model_name, data in model_data.items():
            
            # Search # find the best hyperparamters
            GS = GridSearchCV(
                estimator = data['model'],
                param_grid = data['search space'],
                #n_jobs=2,
                scoring = 'accuracy',
                cv = 5,
                verbose = 3
            )
            
            GS.fit(scaled_x_train, dataset['y_train'])
            
            val_pred = GS.predict(scaled_x_val)
            
            score = accuracy_score(dataset['y_val'], val_pred)
            
            model_metrics.loc[len(model_metrics.index)] = [
                dataset_name,
                scaler_name,
                model_name,
                GS.best_params_,
                score
            ]
            

model_metrics

Fitting 5 folds for each of 22 candidates, totalling 110 fits


c:\Users\Kevin\.virtualenvs\Code-CYgrxAwh\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 1/5] END l1_ratio=0.0, max_iter=10, penalty=elasticnet, solver=saga;, score=0.696 total time=   0.4s


c:\Users\Kevin\.virtualenvs\Code-CYgrxAwh\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 2/5] END l1_ratio=0.0, max_iter=10, penalty=elasticnet, solver=saga;, score=0.693 total time=   0.2s


...

c:\Users\Kevin\.virtualenvs\Code-CYgrxAwh\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Kevin\.virtualenvs\Code-CYgrxAwh\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 4/5] END l1_ratio=1.0, max_iter=10, penalty=elasticnet, solver=saga;, score=0.729 total time=   0.1s


c:\Users\Kevin\.virtualenvs\Code-CYgrxAwh\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 5/5] END l1_ratio=1.0, max_iter=10, penalty=elasticnet, solver=saga;, score=0.728 total time=   0.1s
[CV 1/5] END l1_ratio=1.0, max_iter=100, penalty=elasticnet, solver=saga;, score=0.731 total time=   0.5s
[CV 2/5] END l1_ratio=1.0, max_iter=100, penalty=elasticnet, solver=saga;, score=0.720 total time=   0.1s
[CV 3/5] END l1_ratio=1.0, max_iter=100, penalty=elasticnet, solver=saga;, score=0.719 total time=   0.2s
[CV 4/5] END l1_ratio=1.0, max_iter=100, penalty=elasticnet, solver=saga;, score=0.729 total time=   0.2s
[CV 5/5] END l1_ratio=1.0, max_iter=100, penalty=elasticnet, solver=saga;, score=0.728 total time=   0.3s


,Dataset,Scaling,Model,Hyper params,Accuracy
0,df1,minmax,LogisticRegression,"{'l1_ratio': 0.4, 'max_iter': 10, 'penalty': '...",0.705352
1,df1,standard,LogisticRegression,"{'l1_ratio': 0.0, 'max_iter': 10, 'penalty': '...",0.705205
2,df2,minmax,LogisticRegression,"{'l1_ratio': 1.0, 'max_iter': 10, 'penalty': '...",0.503519
3,df2,standard,LogisticRegression,"{'l1_ratio': 0.4, 'max_iter': 100, 'penalty': ...",0.503372


In [ ]:
# använda GridSearchCV() och välja lämplig evalueringsmetric (accurancy)
# gör prediction på valideringsdata
# beräkna och spara evaluation score för ditt valda metric
# checka bästa parametrarna för respektive modell

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(df1_x_train, df1_y_train)
y_pred = clf.predict(df1_x_test)




# print(classification_report(df1_y_test, y_pred))
# cm = confusion_matrix(df1_y_test, y_pred)
# ConfusionMatrixDisplay(cm).plot();


c:\Users\Kevin\.virtualenvs\Code-CYgrxAwh\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
